# Function Calling

Code inspired by [Openai Cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

## Function Calling Lifecycle

| ![fc-diagram](function-calling-diagram-resized.png) | 
|:--:| 
| *The lifecycle of a function call. Image from [OpenAI](https://platform.openai.com/docs/guides/function-calling/lifecycle).* |

In [1]:
from dotenv import load_dotenv
load_dotenv();

from openai import OpenAI
from formatting import pprint_messages
GPT_MODEL = "gpt-4o"
client = OpenAI()

In [2]:
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [36]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today                                                                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please provide the location for which you'd like to know the weather today,           │
│  including the city and state or country?                                                        │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [37]:
messages.append({"role": "user", "content": "I'm in Santiago, Chile."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today                                                                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please provide the location for which you'd like to know the weather today,           │
│  including the city and state or country?                                                        │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  I'm in Santiago, Chile.                                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius"}',                         │
│  name='get_current_weather')                                                                     │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [39]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago, Chile over the next x days?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)


╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days (x) you would like the weather forecast for in      │
│  Santiago, Chile?                                                                                │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [40]:
messages.append({"role": "user", "content": "3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days (x) you would like the weather forecast for in      │
│  Santiago, Chile?                                                                                │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  3 days                                                                                          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius","num_days":3}',            │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [41]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Buenos Aires, Argentina."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Give me a weather report for Buenos Aires, Argentina.                                           │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Buenos Aires, Argentina","format":"celsius","num_days":5}',    │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [45]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago and Buenos Aires over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    for tool_call in assistant_message.tool_calls:
        messages.append({"role": "function", "content": f"{tool_call.function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago and Buenos Aires over the next 4 days          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Santiago, Chile", "format": "celsius", "num_days": 4}',       │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Buenos Aires, Argentina", "format": "celsius", "num_days":    │
│  4}', name='get_n_day_weather_forecast')                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯